In [47]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from database import get_database , save_to_database
from extract_info import extract_film_info, extract_newoffer_info
from extract_detail import *

In [48]:
# Khởi tạo trình điều khiển Chrome
service = Service()
driver = webdriver.Chrome(service=service)

In [49]:
driver.get("https://www.cgv.vn/default/cinox/site/cgv-hung-vuong-plaza")

In [51]:
def extract_one_day(driver):
    film_data = []
    film_elements = driver.find_elements(By.CLASS_NAME, "film-list")

    for film in film_elements:
        # Lấy tên phim & link chi tiết phim
        title_element = film.find_element(By.CSS_SELECTOR, ".film-label h3 a")
        film_title = title_element.text.strip()
        film_link = title_element.get_attribute("href")

        # Lấy link ảnh poster phim
        poster_element = film.find_element(By.CSS_SELECTOR, ".film-poster img")
        poster_url = poster_element.get_attribute("src")

        # Lấy định dạng chiếu (VD: 2D, Lồng Tiếng)
        film_formats = [fmt.text.strip() for fmt in film.find_elements(By.CLASS_NAME, "film-screen")]

        # Lấy danh sách suất chiếu & link đặt vé
        showtimes = []
        showtime_elements = film.find_elements(By.CSS_SELECTOR, ".film-showtimes li a")
        for showtime in showtime_elements:
            time_text = showtime.find_element(By.TAG_NAME, "span").text.strip()
            ticket_link = showtime.get_attribute("href")
            showtimes.append({"time": time_text, "link": ticket_link})
        # Lưu thông tin phim vào danh sách
        film_data.append({
            "title": film_title,
            "detail_link": film_link,
            "poster_url": poster_url,
            "formats": film_formats,
            "showtimes": showtimes
        })

    return film_data

In [53]:
from datetime import datetime
def extract_all_days(driver):
    # Tìm tất cả thẻ ngày chiếu phim
    date_elements = driver.find_elements(By.CSS_SELECTOR, "li.day.cgv-onlyone")

    all_film_data = {}
    consecutive_empty_days = 0  # Đếm số ngày liên tiếp không có dữ liệu

    for date in date_elements:
        try:
            # Lấy thông tin ngày, tháng, năm
            full_date = date.get_attribute("id").replace("cgv", "")  # Lấy ngày định dạng YYYYMMDD

            # Chuyển đổi thành định dạng "dd-mm-yyyy"
            date_obj = datetime.strptime(full_date, "%Y%m%d")
            formatted_date = date_obj.strftime("%d-%m-%Y")  # Chuyển thành dd-mm-yyyy

            print(f"📅 Đang lấy dữ liệu cho ngày: {formatted_date}")

            # Click vào ngày đó để load nội dung mới
            driver.execute_script("arguments[0].click();", date)

            # Chờ trang cập nhật dữ liệu mới sau khi click
            time.sleep(2)

            # Gọi hàm thu thập dữ liệu phim
            daily_film_data = extract_one_day(driver)

            if not daily_film_data:
                consecutive_empty_days += 1
                print(f"⚠️ Ngày {formatted_date} không có dữ liệu. Số ngày trống liên tiếp: {consecutive_empty_days}")

                # Nếu 2 ngày liên tiếp không có dữ liệu, dừng chương trình
                if consecutive_empty_days >= 2:
                    print("❌ Đã gặp 2 ngày liên tiếp không có dữ liệu, dừng chương trình.")
                    break
            else:
                all_film_data[formatted_date] = daily_film_data
                consecutive_empty_days = 0  # Reset bộ đếm khi có dữ liệu

        except Exception as e:
            print(f"❌ Lỗi khi lấy dữ liệu ngày {formatted_date}: {e}")

    return all_film_data


In [55]:
from selenium.webdriver.common.by import By

def extract_theater_info(driver):
    try:
        # Lấy tiêu đề rạp
        title_element = driver.find_element(By.CLASS_NAME, "page-title.theater-title")
        title = title_element.find_element(By.TAG_NAME, "h3").text.strip()

        # Lấy thông tin rạp
        theater_info = driver.find_element(By.CLASS_NAME, "theater-infomation")
        address = theater_info.find_element(By.CLASS_NAME, "theater-address").text
        fax = theater_info.find_element(By.CSS_SELECTOR, ".fax .fax-input").text
        hotline = theater_info.find_element(By.CSS_SELECTOR, ".hotline .fax-input").text
        map_link = theater_info.find_element(By.CSS_SELECTOR, ".location a").get_attribute("href")

        # Lấy danh sách ảnh rạp
        theater_images = driver.find_element(By.CLASS_NAME, "theater-thumb-image")
        img_elements = theater_images.find_elements(By.TAG_NAME, "img")
        image_urls = [img.get_attribute("src") for img in img_elements]
        film_schedule = extract_all_days(driver)
        # Trả về dữ liệu dưới dạng dictionary
        return {
            "title": title,
            "address": address,
            "fax": fax,
            "hotline": hotline,
            "map_link": map_link,
            "image_urls": image_urls,
            "film_schedule" : film_schedule
        }
    except Exception as e:
        print(f"❌ Lỗi khi trích xuất thông tin rạp: {e}")
        return None

In [56]:
data_info = extract_theater_info(driver)
print(data_info)

📅 Đang lấy dữ liệu cho ngày: 16-03-2025
📅 Đang lấy dữ liệu cho ngày: 17-03-2025
📅 Đang lấy dữ liệu cho ngày: 18-03-2025
📅 Đang lấy dữ liệu cho ngày: 19-03-2025
📅 Đang lấy dữ liệu cho ngày: 20-03-2025
📅 Đang lấy dữ liệu cho ngày: 21-03-2025
📅 Đang lấy dữ liệu cho ngày: 22-03-2025
📅 Đang lấy dữ liệu cho ngày: 23-03-2025
⚠️ Ngày 23-03-2025 không có dữ liệu. Số ngày trống liên tiếp: 1
📅 Đang lấy dữ liệu cho ngày: 24-03-2025
📅 Đang lấy dữ liệu cho ngày: 25-03-2025
⚠️ Ngày 25-03-2025 không có dữ liệu. Số ngày trống liên tiếp: 1
📅 Đang lấy dữ liệu cho ngày: 26-03-2025
⚠️ Ngày 26-03-2025 không có dữ liệu. Số ngày trống liên tiếp: 2
❌ Đã gặp 2 ngày liên tiếp không có dữ liệu, dừng chương trình.
{'title': 'CGV Hùng Vương Plaza', 'address': 'Tầng 7 | Hùng Vương Plaza, 126 Hồng Bàng, Phường 12, Quận 5, TP. Hồ Chí Minh.', 'fax': '+84 4 6 275 5240', 'hotline': '1900 6017', 'map_link': 'https://www.google.com/maps/embed?pb=!1m18!1m12!1m3!1d1959.8605272140667!2d106.66259586811066!3d10.755968268574438!

In [64]:
def extract_site_info(driver, city_id):
    try:
        city_element = driver.find_element(By.ID, city_id)
        driver.execute_script("arguments[0].click();", city_element)
        print(f"✅ Đã chuyển sang khu vực {city_id} để tiếp tục thu thập dữ liệu.")
        time.sleep(2)
    except Exception as e:
        print(f"⚠️ Không thể click vào {city_id}: {e}")

def extract_all_sites(driver):
    try:
        city_elements = driver.find_elements(By.CSS_SELECTOR, "span[id^='cgv_site_']")
        city_ids = [city.get_attribute("id") for city in city_elements]
        print("📌 Danh sách các thành phố có thể thu thập:", city_ids)
        return city_ids
    except Exception as e:
        print(f"⚠️ Lỗi khi lấy danh sách thành phố: {e}")
        return []

def extract_all_theaters(driver):
    city_ids = extract_all_sites(driver)
    for city_id in city_ids:
        extract_site_info(driver, city_id)
        time.sleep(2)
        theater_data = extract_theater_info(driver)
        if theater_data:
            print(theater_data)


In [65]:
data = extract_all_theaters(driver)
print(data)

📌 Danh sách các thành phố có thể thu thập: ['cgv_site_004', 'cgv_site_008', 'cgv_site_021', 'cgv_site_023', 'cgv_site_024', 'cgv_site_030', 'cgv_site_001', 'cgv_site_018', 'cgv_site_027', 'cgv_site_028', 'cgv_site_032', 'cgv_site_037', 'cgv_site_005', 'cgv_site_025', 'cgv_site_020', 'cgv_site_022', 'cgv_site_035', 'cgv_site_014', 'cgv_site_013', 'cgv_site_036', 'cgv_site_026', 'cgv_site_016', 'cgv_site_017', 'cgv_site_003', 'cgv_site_019', 'cgv_site_015', 'cgv_site_006', 'cgv_site_010', 'cgv_site_012', 'cgv_site_031', 'cgv_site_033', 'cgv_site_034', 'cgv_site_038', 'cgv_site_044', 'cgv_site_039', 'cgv_site_043', 'cgv_site_042', 'cgv_site_045', 'cgv_site_029', 'cgv_site_046', 'cgv_site_047', 'cgv_site_054', 'cgv_site_055', 'cgv_site_056', 'cgv_site_051', 'cgv_site_072', 'cgv_site_057', 'cgv_site_060', 'cgv_site_053', 'cgv_site_065', 'cgv_site_062', 'cgv_site_071', 'cgv_site_061', 'cgv_site_048', 'cgv_site_067', 'cgv_site_073', 'cgv_site_074', 'cgv_site_063', 'cgv_site_064', 'cgv_site_07

KeyboardInterrupt: 